

## Прогнозирование стоимости автомобиля по характеристикам


*Помним, что по условию соревнования, нам нужно самостоятельно собрать обучающий датасет.*

К сожалению, датасет собрать не удалось в достаточном объеме. Однако те данные, которые были собраны (более свежие данные) не подошли для обучения модели с целью достижения наиболее высокого результата, скорее всего из-за разницы цен и неоднородности данных в обучающей выборке и тестовой выборке на kaggle. Т.е. марки и модели, которые оказались в обучающей выборке, отсутствовали в тестовой и наоборот. Поэтому было принято решение не использовать свои данные, а взять датасет из baseline: он лучше всего подходит для модели, которая делает предсказание для тестовой выборки соревнований. 

Если же мы говорим об универсальной модели, пригодной для промышленного использования, то, конечно, нужно собирать полный набор данных: чем шире и разнообразнее он будет, тем лучше.


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt
from matplotlib import gridspec
import seaborn as sns
import pylab

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier


In [2]:
#показывать dataframe без ограничения количества столбцов и 100 строк по умолчанию
pd.options.display.max_rows = 100
pd.options.display.max_columns = None

print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

In [3]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [4]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [5]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [6]:
#функция для первичного анализа категориальных столбцов
def plot_str(df,col):

    print('Распределение для столбца (не числовой):', col)
    fig,ax=plt.subplots(figsize=(10,5))
    sns.countplot(x=df.loc[:,col], ax=ax)
    plt.show()
#поиск пустых  Nan значений в символьном  столбце, расчет процента потерянных значений
    n=100-(df[col].count()/df.shape[0]*100)
    print('уникальных значений ', len(df[col].dropna().unique()))
    print ('пустых значений,%', round(n,2))
    #df.default.unique()

# Setup

In [7]:
VERSION    = 16
DIR_TRAIN  = '../input/parsing-all-moscow-auto-ru-09-09-2020/' # подключил к ноутбуку внешний датасет
DIR_TEST   = '../input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20   # 20%

# Data

In [9]:
!ls '../input'

In [ ]:
train = pd.read_csv(DIR_TRAIN+'all_auto_ru_09_09_2020.csv') # датасет для обучения модели
bigtrain = pd.read_csv('../input/data-add/data1.csv')
mtrain=train.copy()

#train_add = pd.read_csv('../input/data-add/data1.csv')
test = pd.read_csv(DIR_TEST+'test.csv')
mtest=test.copy()

mtest.rename(columns={'model_name': 'model'}, inplace=True)
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

In [11]:
test.info()

С точки зрения прогнозирования стоимости автомобиля интерес вызывают следующие признаки:

- 'bodyType' - тип кузова автомобиля, 
- 'brand', - производитель 
- 'fuelType' - тип топлива
- 'modelDate' - дата модели
- 'numberOfDoors' - количество дверей 
- 'productionDate' - дата производства
- 'vehicleTransmission' - вид трансмиссии 
- 'engineDisplacement' - объем двигателя 
- 'enginePower'- мощность двигателя
- 'mileage'- пробег 
- 'Привод'- привод 
- 'Руль' - руль
- 'ПТС' - наличеие ПТС 
- 'model' - модель

Всего 5 числовых признаков : 'modelDate', 'productionDate',  'engineDisplacement', 'enginePower', 'mileage'.
Остальные признаки - категориальные. 
Поэтому будем проводить моделирование в 2 вариантах: 
1. CatBoost, поскольку для ее использования не требуется делать hot encoding, подбор гиперпараметров по сетке.
2. Построение разрженной матрицы с помощью hot ecoding и тестирование различных ML библиотек.

Выбор модели с наилучшим результатом


**1. Исследуем признак - *'vehicleTransmission'***

Теоретически вид трансмиссии должен влиять на цену автомобиля. Данные в train и test распределены примерно одинаково, однако требуется привести наименование трансмисси к одному виду.Пустых значений нет, признакк категориальный можем использовать сразу после преобразования.

In [12]:
mtest['vehicleTransmission'].value_counts()
plot_str(mtest,'vehicleTransmission')
plot_str(mtrain,'vehicleTransmission')

In [13]:
mtest.dropna(subset=['vehicleTransmission'], inplace=True)
#приводим данные  train и test к одинаковым занчениям

mtest.loc[mtest['vehicleTransmission'].str.contains("робот"),'vehicleTransmission'] =  "ROBOT"
mtest.loc[mtest['vehicleTransmission'].str.contains("мех"),'vehicleTransmission'] =  "MECHANICAL"
mtest.loc[mtest['vehicleTransmission'].str.contains("автомат"),'vehicleTransmission'] =  "AUTOMATIC"
mtest.loc[mtest['vehicleTransmission'].str.contains("вариат"),'vehicleTransmission'] =  "VARIATOR"
mtest.vehicleTransmission.value_counts()

2. **Исследуем признак *'fuelType'***

In [14]:
mtest['fuelType'].value_counts()
plot_str(mtest,'fuelType')
plot_str(mtrain,'fuelType')
#mtest.info()

Пустых значений нет, можем включить признак в обучающую выборку без изменеия. Есть одна строка с указанием топлива "универсал". Смотрим что это за автомобиль и изменим значение вида топлива для этой строки на подходящее.

In [15]:
mtrain[mtrain['fuelType']=='универсал']

Эта строка подлежит удалению, поскольку основные поля имеют значение NaN.

**3. Признак *'numberOfDoors'*** 

Количество дверей в автомобиле. Спорно, но возможно будет влиять на стоимость авто, поскольку косвенно подтверждает модель.
проверим, улучшится ли оценка предсказания. Признак категориальный.


In [16]:
mtest[ 'numberOfDoors' ].value_counts()
plot_str(mtest, 'numberOfDoors')
plot_str(mtrain, 'numberOfDoors')

Видим какое-то кол-во авто с 0 дверей.
Посмотрим, что это за объявления:



In [17]:
mtrain[mtrain[ 'numberOfDoors' ]==0]

1 запись. MERCEDES SIMPLEX Это на самом деле старинный раритет без дверей. Поскольку в тестовой выборке есть такой же автомобиль, оставляем запись

In [18]:
mtest[mtest['model']=='SIMPLEX']

4. **Исследуем признак *'engineDisplacement'***: 

Признак содержит огромное количество ошибок, данные разнородные, совершенно не отражают заявленное содержание (объем двигателя). Как-либо его "причесать" не вижу никакой возможности. Удалять строки с ошибками тоже нет смысла, так как это слиьно уменьшит обучающую выборку. Исключаем из фич. По идее требуется заново спарсить значение, но в рамках данной работы сделать это невозможно из-за недостатка времени, поэтому просто удаляем, чтобы не шумел при обучении модели.

In [19]:
print('Распределение для столбца (не числовой):', ' engineDisplacement')
fig,ax=plt.subplots(figsize=(10,70))
sns.countplot(y=mtrain.loc[:,'engineDisplacement'], ax=ax)
plt.show()

In [20]:
#оставляем только данные в обучающей и тестовой выборке, где цена информативна: больше нуля
mtrain=mtrain[mtrain['price']>0]
train=train[train['price']>0]


Определяем поля - кандидаты для включения в модель

In [21]:
#формируем предварительный список нужных столбцов 
mcolumns=['bodyType', 'brand', 'fuelType', 'modelDate',
       'numberOfDoors', 'productionDate',
       'vehicleTransmission', 'engineDisplacement', 'enginePower',
       'mileage', 'Привод', 'Руль',
       'ПТС', 'price', 
       'model']


In [22]:
#### Удаляем строки с незаполненными значениями важных признаков

mtrain.dropna(subset=['bodyType'], inplace=True)
mtrain.dropna(subset=['Привод'], inplace=True)

**5. 'bodyType'** 

Выяснено, что есть большое разнообразие типов кузовов, однако многие, представленные в обоих датасетах по смыслу повторяют друг друга, но описаны по-разному. Необходимо привести значения к общему виду.

In [23]:
#приводим тип кузова к общим значениям
def rename_body (m):
    m['bodyType']=m['bodyType'].apply(lambda x: str.lower(x))
    m['bodyType']=m['bodyType'].str.replace(' ','')
    m.loc[m['bodyType'].str.contains("внедорожник3"),'bodyType'] =  "Внедорожник3"
    m.loc[m['bodyType'].str.contains("внедорожник5"),'bodyType'] =  "Внедорожник5"
    m.loc[m['bodyType'].str.contains("седанlimousine"),'bodyType'] =  "СеданLong"
    m.loc[m['bodyType'].str.contains("седанlong"),'bodyType'] = "СLong"
    m.loc[m['bodyType'].str.contains("седан-хардтоп"),'bodyType'] = "хардтоп"
    m.loc[m['bodyType'].str.contains("седан"),'bodyType'] = "Седан"    
    m.loc[m['bodyType'].str.contains( "минивэн"),'bodyType'] = "Минивэн"
    m.loc[m['bodyType'].str.contains( "компактвэн"),'bodyType'] = "Минивэн"
    m.loc[m['bodyType'].str.contains("универсал5"),'bodyType']  =  "Универсал5"
    m.loc[m['bodyType'].str.contains("хэтчбек5"),'bodyType'] = "Хэтчбек5"
    m.loc[m['bodyType'].str.contains("хэтчбек4"),'bodyType'] = "Хэтчбек5"
    m.loc[m['bodyType'].str.contains("пикапдв"),'bodyType'] = "Пикап2"
    m.loc[m['bodyType'].str.contains("пикапод"),'bodyType'] = "Пикап1"
    m.loc[m['bodyType'].str.contains("пикаппол"),'bodyType'] = "Пикап1_5"
    m.loc[m['bodyType'].str.contains("фургон"),'bodyType'] = "Фургон"
    m.loc[m['bodyType'].str.contains("родстер"),'bodyType'] = "Родстер"
    m.loc[m['bodyType'].str.contains("тарга"),'bodyType'] = "Родстер"
    m.loc[m['bodyType'].str.contains("кабриолет"),'bodyType'] = "Кабриолет"
    m.loc[m['bodyType'].str.contains("хэтчбек3"),'bodyType'] = "Хэтчбек3"
    m.loc[m['bodyType'].str.contains("лифтбек"),'bodyType'] = "Лифтбек"
    m.loc[m['bodyType'].str.contains("купе "),'bodyType'] = "Купе"
    m.loc[m['bodyType'].str.contains("микровэн"),'bodyType'] = "Микровэн"
    





In [ ]:
#применяем измеения к тренировочной и тестовой выборке.
rename_body (mtrain)
rename_body (mtest)

**6. 'brand'**

Цель работы - построение модели для получения наилучшего результата на выборке в kaggle, поэтому есть смысл ограничить обучающую выбрку только брендами, которые присутствуют в тестовой выборке.

Для построения настоящей моедли для промышленного использования это, конечно же, делать не стоит.

In [24]:
#Определили список брендов. Только на этих брендах есть смысл обучать модель
brand_list=mtest.brand.unique()

In [25]:
brand_list

In [26]:
#Сократим обучающую выбрку только до брендов, которые есть в тестовой выборке.
mtrain['brand']=mtrain['brand'].apply(lambda x:str.upper(x))
mtrain['bodyType']=mtrain['bodyType'].str.replace(' ','')

mtrain=mtrain[mtrain['brand'].isin (brand_list)]
mtrain=mtrain[mtrain['price']>0]


**7. Числовые столбцы: посмотрим на распределение и выбросы**

Оставляем 4 числовых столбца: 'productionDate','mileage','enginePower','modelDate'

In [28]:

#посмотрим на чиловые столбцы и прологарифмируем их (эксперимент)
cols_to_ln=['productionDate','mileage','enginePower','modelDate']
print ('До логарифмирования')
fig, axes = plt.subplots(1, 4, figsize=(20,5))

for col, i in zip(cols_to_ln, range(4)):   
    sns.histplot(mtrain[col], kde=False, ax=axes.flat[i])
    
plt.show()



In [29]:
print ('После логарифмирования')
fig, axes = plt.subplots(1, 4, figsize=(20,5))

a=np.log(mtrain[cols_to_ln])
for col, i in zip(cols_to_ln, range(4)):#,'mileage','productionDate'
    
    sns.histplot(a[col], kde=False, ax=axes.flat[i])
    
pylab.show()

In [30]:
mtrain['mileage']=np.log(mtrain['mileage'])
mtest['mileage']=np.log(mtest['mileage'])
mtrain['productionDate']=np.log(mtrain['productionDate'])
mtest['productionDate']=np.log(mtest['productionDate'])
#Для тестовой выборки почистим столбец и преобразуем его в число
mtest['enginePower']=mtest['enginePower'].apply(lambda x: int(x[:-3].strip()))
mtrain['enginePower']=np.log(mtrain['enginePower'])
mtest['enginePower']=np.log(mtest['enginePower'])
mtrain['modelDate']=np.log(mtrain['modelDate'])
mtest['modelDate']=np.log(mtest['modelDate'])
mtest=mtest.replace(-np.Inf, 0)
mtest=mtest.replace(np.NINF, 0)
mtest=mtest.replace(np.Inf, 0)
mtrain=mtrain.replace(-np.Inf, 0)
mtrain=mtrain.replace(np.NINF, 0)
mtrain=mtrain.replace(np.Inf, 0)

## Data Preprocessing

In [31]:
train.dropna(subset=['productionDate','mileage','Руль','Привод','enginePower',
                     'modelDate','fuelType','numberOfDoors','vehicleTransmission','model'], inplace=True)
train.dropna(subset=['price'], inplace=True)

mtrain.dropna(subset=['productionDate','mileage','Руль','Привод','enginePower',
                      'modelDate','fuelType','numberOfDoors','vehicleTransmission','model'], inplace=True)
mtrain.dropna(subset=['price'], inplace=True)

mtrain=mtrain[mtrain['price']>0]


In [33]:
columns = ['bodyType', 'brand', 'productionDate',  'mileage','Руль',
           'Привод','enginePower','modelDate','fuelType','numberOfDoors','vehicleTransmission','model']#'engineDisplacement',
df_train = mtrain[columns]
df_test = mtest[columns]

In [34]:
y = mtrain['price']

## Label Encoding

In [35]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
mdata = data.copy()

**8. 'Руль'**
Приведем данные к одинаковым значениям

In [36]:
data['Руль']=data['Руль'].apply(lambda x: 'LEFT' if x =='Левый' else 'RIGHT')

for colum in ['bodyType', 'brand', 'Руль','Привод','fuelType','numberOfDoors','vehicleTransmission','model']:
    data[colum] = data[colum].astype('category').cat.codes #'engineDisplacement',


**9. 'model'** 
Есть смысл сравнить списки моделей из обучающей и тестовой выборок, сравнить 2 множества, посмотерть пересечение. Исключений не очень много, признак оставим, модель сильно влияет на стоимость авто.

In [53]:
tr=mtrain.model.unique().tolist()
ts=mtest.model.unique().tolist()
result=list(set(ts) - set(tr))
sorted(result)

In [54]:
data.sample(5)

In [39]:
X = data.query('sample == 1').drop(['sample'], axis=1)
X_sub = data.query('sample == 0').drop(['sample'], axis=1)

In [41]:
union_data = mtest.append(mtrain, sort=False).reset_index(drop=True) # объединяем

## Train Split

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

# Model 1: Создадим первую модель 
Эта модель будет предсказывать среднюю цену по отобранным ранее признакам.



In [43]:
tmp_train = X_train.copy()
tmp_train['price'] = y_train

# # Model 1 : CatBoost

У нас в данных практически все признаки категориальные. Специально для работы с такими данными была создана очень удобная библиотека CatBoost от Яндекса. [https://catboost.ai](http://)     
На данный момент **CatBoost является одной из лучших библиотек для табличных данных!**

## Fit

In [45]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, y_train,
         #cat_features=cat_features_ids,
         eval_set=(X_test, y_test),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

model.save_model('catboost_single_model_baseline.model')

In [47]:
# оцениваем точность
predict = model.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

На обработанных и отобранных признаках получили ошибку в 14%. 

### Log Traget
Попробуем взять таргет в Log - это позволит уменьшить влияние выбросов на обучение модели (используем для этого np.log и np.exp).    
Также в этой модели используем параметры, подобранные по сетке. 

In [48]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         depth = 10, l2_leaf_reg = 7, learning_rate = 0.1)
model.fit(X_train, np.log(y_train),
         #cat_features=cat_features_ids,
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

model.save_model('catboost_single_model_2_baseline.model')

In [ ]:
#Оптимизируем гиперпараметры
#model_random = CatBoostRegressor()
#grid = {'learning_rate': [0.1, 0.2,0.3],
#        'depth': [ 6,8,10,12],
#        'l2_leaf_reg': [3,5,  7,9]}

#search_result = model_random.grid_search(grid, X=X_train, y=y_train, verbose=3, plot=True)
#print(search_result['params'])

In [ ]:
#search_result['params']
#grid_cv = model_selection.GridSearchCV(CatBoostRegressor(), parameters_grid, scoring = 'MAPE', cv = cv)
#grid_cv.fit(X_train, X_test)

In [ ]:
#model_random.best_params_

In [49]:
predict_test = np.exp(model.predict(X_test))
predict_submission = np.exp(model.predict(X_sub))

In [50]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Однако когда делаем submit, ошибка выратстает до 12.56.

## Пробуем другие ML модели: 
1.Подготовка train: labelIncoding


In [51]:
mdata.info()
cat_cols = ['bodyType', 'brand', 'Руль','Привод','fuelType','numberOfDoors','vehicleTransmission','model']#
num_cals=['productionDate','mileage','enginePower', 'modelDate']

Построим матрицу корреляций по чилосвым столбцам

In [52]:
plt.rcParams['figure.figsize'] = (15,10)

matrix = np.triu(mdata[mdata['sample'] == 0].corr())
sns.heatmap(mdata.corr(), annot=False, mask=matrix, cmap= 'coolwarm')

Видим, что дата выпуска сильно коррелирует с датой модели. Однако, при удалении одного из признаков ощутимо снижаетсяя оценка. Оставляем оба признака и позже проверим, возможно, из 2 признаков можно сделать 1.
Также есть достаточно сильная обратная корелляция этих признаков с пробегом.

In [55]:
#Определяем dummy-переменные
dummies = pd.get_dummies(mdata[cat_cols])

In [56]:
mdata_dum = pd.concat([mdata, dummies], axis=1)

In [57]:
mdata_dum.sample(5)

In [58]:
mdata_d = mdata_dum.drop(columns=cat_cols, axis=1)

In [60]:
mdata_d.sample(5)

In [61]:
XX = mdata_d.query('sample == 1').drop(['sample'], axis=1)
XX_sub = mdata_d.query('sample == 0').drop(['sample'], axis=1)

# Splitting the data
XX_train, X_val, yy_train, y_val = train_test_split(XX, y,  test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

In [62]:
yy_train=np.log(yy_train)

In [63]:
XX_train=XX_train.astype(float)
y_train=y_train.astype(float)
XX_train=XX_train.replace(-np.Inf, 0)
XX_train=XX_train.replace(np.NINF, 0)
XX_train=XX_train.replace(np.Inf, 0)
y_train=y_train.replace(-np.Inf, 0)
y_train=y_train.replace(np.NINF, 0)
y_train=y_train.replace(np.Inf, 0)
yy_train=yy_train.replace(-np.Inf, 0)
yy_train=yy_train.replace(np.NINF, 0)
yy_train=yy_train.replace(np.Inf, 0)

### XGBRegressor. 
Гиперпараметры были подобраны опытным путем, поскольку подбор по сетке занимал очень много времени.

In [64]:

xg_reg = xgb.XGBRegressor ( colsample_bytree= 0.7, 
                            learning_rate= 0.03, 
                            max_depth= 12, 
                            min_child_weight = 4, 
                            n_estimators = 500, 
                            nthread = 4, 
                            
                            subsample = 0.7)

#(objective='reg:squarederror', colsample_bytree=0.5,
#                          learning_rate=0.05, max_depth=12, alpha=1,
#                          n_estimators=1000)
xg_reg.fit(XX_train, yy_train)

In [65]:
predict_xg_reg = xg_reg.predict(X_val)
predict_xg_reg_sub = np.exp(xg_reg.predict(XX_sub))
#np.exp(model.predict(X_sub))
#display(predict)
#display(y_test)
print(f"Точность модели по метрике MAPE: {(mape(y_val, np.exp(predict_xg_reg)))*100:0.2f}%")

Submit - ошибка 13.46%

### Случайный лес

In [66]:
#  Random Forest
rf = ExtraTreesRegressor(n_estimators=300, random_state=RANDOM_SEED, n_jobs=-1,
                         bootstrap=True, verbose=1)
rf.fit(XX_train, yy_train)

In [67]:
pred_rf = rf.predict(X_val)
pred_rf_sub=np.exp(rf.predict(XX_sub))

In [69]:
MAPE = mape(y_test, pred_rf)
print(f"Точность модели по метрике MAPE: {(mape(y_val, np.exp(pred_rf)))*100:0.2f}%")
#print(f'Mean Absolute Percentage Error: {MAPE}')

Submit - ошибка 12.42%

### GradientBoostingRegressor

In [74]:
gbr = GradientBoostingRegressor(loss ='ls',n_estimators = 900, max_depth=10)
gbr.fit (XX_train, yy_train)

pred_gbr = gbr.predict(X_val)
pred_gbr_sub= np.exp(gbr.predict(XX_sub))

In [75]:
print(f"Точность модели по метрике MAPE: {(mape(y_val, np.exp(pred_gbr)))*100:0.2f}%")

In [ ]:
Submit - ошибка 12.12%

In [72]:
#GradientBoosting c на  датасете, подготовленном для catboost
gbr = GradientBoostingRegressor(loss ='ls', max_depth=10, n_estimators = 900)
gbr.fit (X_train, y_train)

In [73]:
pred_gbr_1 = gbr.predict(X_test)
pred_gbr1_sub= gbr.predict(X_sub)
print(f"Точность модели по метрике MAPE: {(mape(y_test, pred_gbr_1))*100:0.2f}%")

Та же самая модель но на датасете, подготовленном для catboost с кодированием признаков дала ошибку ощутимо выше.

# Submission

In [ ]:
sample_submission['price'] = pred_gbr_sub
sample_submission.to_csv(f'submission.csv', index=False)

In [ ]:
sample_submission

В итоге получили **MAPE 12,12%** 

Есть небольшая разница в размере ошибки между test и submission, которая наименее заметна при использовании  GradientBoostingRegressor.